In [40]:
import tensorflow as tf
import pprint
import pycurl
from io import BytesIO
import json
import time

### Load model from file

In [118]:
sess = tf.Session()
# we want the variable values from from my_tf_model (not working right now)
saver = tf.train.import_meta_graph('models/my_tf_model.meta')
saver.restore(sess, tf.train.latest_checkpoint('models'))

graph = tf.get_default_graph()

def conv_net(x,reuse=a, dropout=0.1610, kernel_size=3, filters=32, hiddenunits=500):
    # Define a scope for reusing the variables
    # with tf.variable_scope('model', reuse=reuse): is this needed?
    conv1    = tf.layers.conv2d(inputs=x,
                                filters=filters,
                                kernel_size=kernel_size,
                                activation=tf.nn.relu)
    maxpool1 = tf.layers.max_pooling2d(inputs=conv1,
                                       pool_size=2,
                                       strides=2)
    conv2    = tf.layers.conv2d(inputs=maxpool1,
                                filters=filters,
                                kernel_size=kernel_size,
                                activation=tf.nn.relu)
    maxpool2 = tf.layers.max_pooling2d(inputs=conv2,
                                       pool_size=2,
                                       strides=2)
    conv3    = tf.layers.conv2d(inputs=maxpool2,
                                filters=filters,
                                kernel_size=kernel_size,
                                activation=tf.nn.relu)
    maxpool3 = tf.layers.max_pooling2d(inputs=conv2,
                                       pool_size=2,
                                       strides=2)
    flat     = tf.contrib.layers.flatten(inputs=maxpool3)
    dropout1 = tf.layers.dropout(inputs=flat,
                                 rate=dropout,
                                 training=True)
    dense1   = tf.layers.dense(inputs=dropout1,
                               units=hiddenunits,
                               activation=tf.nn.relu)

    dropout2 = tf.layers.dropout(inputs=dense1,
                                 rate=dropout,
                                 training=True)
    out = tf.layers.dense(dropout2,
                          units=1)

    return out


INFO:tensorflow:Restoring parameters from models/my_tf_model


### Graph info

In [116]:
print(graph.get_all_collection_keys())
print(len(graph.get_collection('trainable_variables')))

['summaries', 'trainable_variables', 'train_op', 'variables', 'queue_runners', 'update_ops']
730


### Functions

In [85]:
# returns prediction of number of cars from img of dimensions 200x112x3
def count_from_image(img):
    print("build graph")
    c = conv_net(img, reuse=True)
    print("done")
    return c

# returns image from webcam with id
def get_img(webcam_id):
    image = tf.read_file("data/test_image.jpg")
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize_images(image, [112, 200])
    return tf.expand_dims(image,0)

    

### Main program

In [95]:
webcam_id_list = [33] #dummy values
sleep_time = 60 #check every minute
while(1):
    for w in webcam_id_list:
        i = get_img(w)
        g = count_from_image(i)
        sess.run(tf.global_variables_initializer())
        # forward prop the img and recieve prediction
        count = sess.run(g)
        print("Webcam " + str(w) + " has " + str(count[[0]]) + " number of cars")
    time.sleep(sleep_time)

build graph
Tensor("ExpandDims_111:0", shape=(1, 112, 200, 3), dtype=float32)
Tensor("conv2d_82/Relu:0", shape=(1, 110, 198, 32), dtype=float32)
done
INFO:tensorflow:Restoring parameters from models/my_tf_model
Webcam 33 has [[ 34.11844254]] number of cars


KeyboardInterrupt: 